<a href="https://colab.research.google.com/github/ajankowski/hotdog_or_not/blob/main/hot_dog_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning



## Import libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Get the data

In [ ]:
# git clone command downloads the git repository 
# unzip extracts the data 
! git clone https://github.com/ajankowski/hotdog_or_not.git
! unzip hotdog_or_not/test.zip
! unzip hotdog_or_not/train.zip

In [ ]:
# create list of images

img_list = []

for dir in os.listdir('test'):
    for file in os.listdir(os.path.join('test', dir)):
        img_list.append(os.path.join('test', dir, file))

len(img_list)

In [ ]:
# sample images from the list and show it

sample_images = [img_list[i] for i in [0, 100, 300, 400]]

plt.figure(figsize=(8, 8))
for i in range(4):
   plt.subplot(2, 2, i+1)
   plt.imshow(plt.imread(sample_images[i]))
   plt.axis('off')
   plt.tight_layout()

In [ ]:
# Instantiate two image generator classes:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [ ]:
# Define the batch size:
batch_size=32

In [ ]:
# Define the train and test generators: 
classes = ['hot_dog','not_hot_dog']

train_generator = train_datagen.flow_from_directory(
    directory='train',
    target_size=(224, 224),
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

test_generator = test_datagen.flow_from_directory(
    directory='test',
    target_size=(224, 224),
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

# The model

## Load the pre-trained model

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

## Freeze the layers in the base (pretrained) model

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

## Build the neural network

In [ ]:
model = tf.keras.models.Sequential()

# pretrained model
model.add(base_model)

# your own classifier 
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

## Compile the model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training

In [ ]:
model.fit(train_generator, steps_per_epoch=15, 
                    epochs=10, validation_data=test_generator, 
                    validation_steps=15)

In [ ]:
#@title Load sample images
img1_adress = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/delish-210326-blt-hotdog-02-landscape-jg-1622060198.jpg" #@param {type:"string"}
img2_adress = "https://s3.przepisy.pl/przepisy3ii/img/variants/800x0/pizza-grecka.jpg" #@param {type:"string"}
!wget -O test1 $img1_adress
!wget -O test2 $img2_adress

In [ ]:
img_1 = tf.keras.preprocessing.image.load_img('test1', target_size=(224, 224))
img_2 = tf.keras.preprocessing.image.load_img('test2', target_size=(224, 224))

In [ ]:
plt.imshow(img_1)
plt.axis('off')
plt.show()

In [ ]:
pred1 = model.predict(np.expand_dims(np.array(img_1), axis=0))
pred1

In [ ]:
classes[np.argmax(pred1)]

In [ ]:
plt.imshow(img_2)
plt.axis('off')
plt.show()

In [ ]:
pred2 = model.predict(np.expand_dims(np.array(img_2), axis=0))
pred2

In [ ]:
classes[np.argmax(pred2)]

# Let's deploy it

In [ ]:
!pip install gradio -q

In [ ]:
import gradio as gr

In [ ]:
def inference(image):
    img = np.expand_dims(image, axis=0)
    prediction = classes[np.argmax(model.predict(img))]
    return prediction

input = gr.inputs.Image(shape=(224, 224))

interface = gr.Interface(fn=inference, inputs=input, outputs=gr.outputs.Label())

interface.launch(share=True, debug=True)